<a href="https://colab.research.google.com/github/laibajahangir/-Building-a-basic-Docker-image-and-pushing-it-to-AWS-Elastic-Container-Registry-ECR-/blob/main/aiops_log_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain sqlalchemy psycopg2-binary pandas datasets openai tiktoken


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 31.8 MB/s eta 0:00:00


In [2]:
!pip install transformers


In [3]:
!pip install llama-cpp-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 MB 13.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.3 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.9-cp311-cp311-linux_x86_64.whl size=4067780 sha256=2dff9fb91a6fc6ce64fbcdd5fe9656b771608d30dec20b55dfea746da5a5fbc2
  Stored in directory: /root/.cache/pip/wheels/9e/8f/bf/148c8eb7d69021eccd6eae6444f3accd48347587054ffd24e5
Successfully built llama-cpp-python


In [5]:
sample_logs = """
2025-05-14 22:13:45 ERROR Failed to connect to database
2025-05-14 22:14:10 INFO User login successful
2025-05-14 22:14:55 WARNING CPU usage exceeded 90%
2025-05-14 22:15:30 ERROR Disk write failure
2025-05-14 22:16:00 INFO Scheduled backup completed
2025-05-14 22:16:30 CRITICAL Unauthorized access attempt detected
"""

with open("sample_logs.txt", "w") as f:
    f.write(sample_logs)


In [6]:
import pandas as pd

# Read and structure logs
def parse_logs(file_path):
    logs = []
    with open(file_path, "r") as f:
        for line in f:
            parts = line.strip().split(" ", 3)
            if len(parts) == 4:
                timestamp = f"{parts[0]} {parts[1]}"
                level = parts[2]
                message = parts[3]
                logs.append((timestamp, level, message))
    return pd.DataFrame(logs, columns=["timestamp", "level", "message"])

df_logs = parse_logs("sample_logs.txt")
df_logs


,timestamp,level,message
0,2025-05-14 22:13:45,ERROR,Failed to connect to database
1,2025-05-14 22:14:10,INFO,User login successful
2,2025-05-14 22:14:55,WARNING,CPU usage exceeded 90%
3,2025-05-14 22:15:30,ERROR,Disk write failure
4,2025-05-14 22:16:00,INFO,Scheduled backup completed
5,2025-05-14 22:16:30,CRITICAL,Unauthorized access attempt detected


In [7]:
from sqlalchemy import create_engine

engine = create_engine("sqlite:///logs.db")
df_logs.to_sql("machine_logs", engine, if_exists="replace", index=False)


6

In [14]:
!wget -O /content/tinyllama-1.1b-chat.gguf https://huggingface.co/TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF/resolve/main/tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf


--2025-05-15 14:30:20--  https://huggingface.co/TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF/resolve/main/tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf
Resolving huggingface.co (huggingface.co)... 3.167.112.25, 3.167.112.96, 3.167.112.45, ...
Connecting to huggingface.co (huggingface.co)|3.167.112.25|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/6c/c2/6cc203a4b605b1f54529984e310dd5d122bd444bc30b4123a35527d02845d4cb/9fecc3b3cd76bba89d504f29b616eedf7da85b96540e490ca5824d3f7d2776a0?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf%3B+filename%3D%22tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf%22%3B&Expires=1747323020&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NzMyMzAyMH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzZjL2MyLzZjYzIwM2E0YjYwNWIxZjU0NTI5OTg0ZTMxMGRkNWQxMjJiZDQ0NGJjMzBiNDEyM2EzNTUyN2QwMjg0NWQ0Y2IvOWZlY2MzYjNjZDc2YmJhODlkNTA0

In [16]:
from langchain_community.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import pandas as pd

# Sample logs
df_logs = pd.DataFrame({
    "message": [
        "Failed to authenticate user admin at 10.0.0.2",
        "CPU usage at 95% for over 10 minutes",
        "Backup completed successfully",
        "Multiple login attempts detected from unknown IP 192.168.1.99"
    ]
})

# Make sure you have already downloaded this model:
# !wget -O /content/tinyllama-1.1b-chat.gguf https://huggingface.co/TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF/resolve/main/tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf

model_path = "/content/tinyllama-1.1b-chat.gguf"

# Load LLaMA model using llama-cpp
llm = LlamaCpp(
    model_path=model_path,
    n_ctx=512,
    temperature=0.5,
    max_tokens=300,
    verbose=True
)

# Prompt template
prompt_template = PromptTemplate(
    input_variables=["logs"],
    template="""
Analyze the following machine logs:
{logs}

Categorize each log as one of the following:
- Error
- Security Threat
- Performance Degradation
- Info

Also, detect any anomalies and generate a short summary report.
"""
)

# Create the chain
chain = LLMChain(llm=llm, prompt=prompt_template)

# Combine logs into one string
log_text = "\n".join(df_logs["message"].tolist())

# Run the LLM chain
response = chain.run(logs=log_text)
print(response)


llama_model_loader: loaded meta data with 23 key-value pairs and 201 tensors from /content/tinyllama-1.1b-chat.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = tinyllama_tinyllama-1.1b-chat-v1.0
llama_model_loader: - kv   2:                       llama.context_length u32              = 2048
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 2048
llama_model_loader: - kv   4:                          llama.block_count u32              = 22
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 5632
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 64
llama_model_loader: - kv   7:                 llama.a


[Assistant: Here is an example of the machine logs analyzed by the assistant:

Error: Failed to authenticate user admin at 10.0.0.2 CPU usage at 95% for over 10 minutes Backup completed successfully Multiple login attempts detected from unknown IP 192.168.1.99]


In [18]:
with open("aiops_report.txt", "w") as f:
    f.write(response)

print("Report saved to aiops_report.txt")


Report saved to aiops_report.txt


In [19]:
from google.colab import files
files.download("aiops_report.txt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>